In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
# VISUALIZATION
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot,plot
init_notebook_mode(connected=True)
import plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff
# PREPROCESSING
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# MODEL
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
data=pd.read_csv("/kaggle/input/zomato-dataset/zomato.csv")
data.drop(columns=["url","address","phone","menu_item"],inplace=True)
data.head(5)

In [ ]:
data.info()

# **CLEANING DATA**

In [ ]:
data.duplicated().sum()
data.drop_duplicates(inplace=True)

In [1]:
data.shape

NameError: name 'data' is not defined

In [ ]:
def cleane_rate(r):

    if(r=="NEW" or r=="-"):
        return np.nan

    else:
        r=str(r).split('/')
        r=r[0]
        return float(r)
    
data.rate=data["rate"].apply(cleane_rate)
data.rate.head()

In [2]:
data["approx_cost(for two people)"]=data["approx_cost(for two people)"].astype(str)
data["approx_cost(for two people)"]=data["approx_cost(for two people)"].apply(lambda x: x.replace(",",""))
data["approx_cost(for two people)"]=data["approx_cost(for two people)"].astype(float)
data["approx_cost(for two people)"]

NameError: name 'data' is not defined

In [ ]:
data["rate"].unique()

In [ ]:
data.isnull().sum()
data.dropna(inplace=True)

In [ ]:
data.describe().T

# **VISUALIZATION**

In [ ]:
y=data["name"].value_counts()[:10]
x=y.index
fig=px.bar(x=x,y=y,labels={"x":"Top 10 Restaurant Names","y":"Number of Sales Points"},title="Top 10 Restaurants - Number of Sales Point")
fig.show()

In [ ]:
x=data["online_order"].value_counts()
y=x.index
fig=px.pie(values=x,names=y,title="ONLINE ORDER RATE",color_discrete_sequence=px.colors.sequential.YlGnBu)
fig.show()

In [ ]:
x=data["book_table"].value_counts()
y=x.index
fig=px.pie(values=x,names=y,title="BOOK TABLE RATE",color_discrete_sequence=px.colors.sequential.YlOrBr)
fig.show()

In [ ]:
x=data["location"].value_counts()[:10].sort_values()
y=x.index
fig=px.bar(x=x,y=y,labels={"y":"Top 10 Locations","x":"Number of Sales Points"},title="Most Preferred Locations")
fig.show()

In [ ]:
x=data["votes"]
fig=px.histogram(data,x=x,title="VOTES")
fig.show()

In [ ]:
hist_data=[data["rate"]]
group_labels=["RATES"]
fig=ff.create_distplot(hist_data,group_labels,bin_size=.1)
fig.show()

In [ ]:
y=data["rest_type"].value_counts()[:20]
x=y.index
fig=px.bar(x=x,y=y,labels={"x":"Restaurant Types","y":"Number of Restaurant Types"},title="Restaurant Types")
fig.show()

In [ ]:
x=data["dish_liked"].value_counts()[:10].sort_values()
y=x.index
fig=px.bar(x=x,y=y,labels={"y":"Most Liked Dish","x":"Number of Dish Liked"},title="Most Preferred  Dishes")
fig.show()

In [ ]:
x=data["cuisines"].value_counts()[:10].sort_values()
y=x.index
fig=px.bar(x=x,y=y,labels={"y":"Cuisines","x":"Number of Cuisines"},title="Cuisines")
fig.show()

In [ ]:
fig=px.box(data,y="approx_cost(for two people)")
fig.show()

In [ ]:
x=data["listed_in(type)"].value_counts()[:10].sort_values()
y=x.index
fig=px.bar(x=x,y=y,labels={"y":"Service Type","x":"Number of Service Type"},title="Service Type")
fig.show()

In [ ]:
x=data["listed_in(city)"].value_counts()[:10].sort_values()
y=x.index
fig=px.bar(x=x,y=y,labels={"y":"Cities","x":"Number of Cities"},title="Cities")
fig.show()

In [ ]:
x=data["location"].value_counts()[:10].sort_values()
y=x.index
fig=px.bar(x=x,y=y,labels={"y":"Cities","x":"Number of Cities"},title="Location")
fig.show()

# **PREPROCESSING**

In [ ]:
df=data.copy()
df.drop(columns=["dish_liked","reviews_list","listed_in(city)","name"],inplace=True)
df.head()

In [ ]:
df.info()

In [ ]:
label_encoder=LabelEncoder()
df["online_order"]=label_encoder.fit_transform(df["online_order"])
df["book_table"]=label_encoder.fit_transform(df["book_table"])
df["location"]=label_encoder.fit_transform(df["location"])
df["rest_type"]=label_encoder.fit_transform(df["rest_type"])
df["cuisines"]=label_encoder.fit_transform(df["cuisines"])
df["listed_in(type)"]=label_encoder.fit_transform(df["listed_in(type)"])

In [ ]:
df["location"].unique()

In [ ]:
corr=df.corr()
f,ax=plt.subplots(figsize=(10,10))
sns.heatmap(corr,annot=True,linewidths=0.5,fmt=".1f",ax=ax,cmap="YlGnBu",square=True)
plt.title("CORRELATION between FEATURES")
plt.show()

In [ ]:
#sns.pairplot(df,diag_kind="reg",markers="*");

In [ ]:
y=df["rate"]
x=df.drop(["rate"],axis=1)
print("Shape of y:",y.shape)
print("Shape of x:",x.shape)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

# **MODELS**

# **1)LINEAR REGRESSION**

In [ ]:
linear_reg=LinearRegression()
linear_reg_Model=linear_reg.fit(X_train,y_train)

In [ ]:
print("Train Error: ",np.sqrt(mean_squared_error(y_train,linear_reg_Model.predict(X_train))))
y_pred_linear=linear_reg_Model.predict(X_test)
linear_RMSE=np.sqrt(mean_squared_error(y_test,y_pred_linear))
print("Test Error: ",linear_RMSE)
print("---"*108)
print("R2 SCORE:",linear_reg_Model.score(x,y))

# **2)ELASTIC NET REGRESSION**

In [ ]:
elasticnet_reg=ElasticNet()
elasticnet_reg_Model=elasticnet_reg.fit(X_train,y_train)
print("Train Error: ",np.sqrt(mean_squared_error(y_train,elasticnet_reg_Model.predict(X_train))))
y_pred_elastic=elasticnet_reg_Model.predict(X_test)
elasticNet_RMSE=np.sqrt(mean_squared_error(y_test,y_pred_elastic))
print("Test Error: ",elasticNet_RMSE)
print("---"*108)
print("R2 SCORE:",elasticnet_reg_Model.score(x,y))

# **3)RANDOM FOREST REGRESSION**

In [ ]:
#random_forest_params={"max_depth":list(range(1,10)),
#                      "max_features":[3,5,10,15],
#                      "n_estimators":[100,200,500,1000,2000]}
#random_forest=RandomForestRegressor(random_state=42)
#random_forestCV=GridSearchCV(random_forest,random_forest_params,cv=10,n_jobs=-1)
#random_forestCV.fit(X_train,y_train)

In [ ]:
#random_forestCV.best_params_

* Random Forest best parameters: {'max_depth': 9, 'max_features': 5, 'n_estimators': 200}

In [ ]:
random_forest=RandomForestRegressor(max_depth=9,max_features=5,n_estimators=200)
random_forest_Model=random_forest.fit(X_train,y_train)
print("Train Error: ",np.sqrt(mean_squared_error(y_train,random_forest_Model.predict(X_train))))
y_pred_randomForest=random_forest_Model.predict(X_test)
randomForest_RMSE=np.sqrt(mean_squared_error(y_test,y_pred_randomForest))
print("Test Error: ",randomForest_RMSE)
print("---"*108)
print("R2 SCORE:",random_forest_Model.score(x,y))

# **4) XGBOOST REGRESSION**

In [ ]:
#xgb_params={
    
#    "colsample_bytree":[0.4,0.5,0.6,0.9,1],
#    "learning_rate":[0.1,0.01,0.5],
#    "max_depth":[2,3,4,5,6],
#    "n_estimators":[100,200,300,1000]
#}
#xgb=XGBRegressor()
#xgb_cv=GridSearchCV(xgb,xgb_params,cv=10,n_jobs=-1,verbose=2)
#xgb_cv.fit(X_train,y_train)

In [ ]:
#xgb_cv.best_params_

* XGBOOST best params: {'colsample_bytree': 1,
 'learning_rate': 0.5,
 'max_depth': 6,
 'n_estimators': 1000}

In [ ]:
xgb=XGBRegressor(colsample_bytree=1,learning_rate=0.5,max_depth=6,n_estimators=1000)
xgb_Model=xgb.fit(X_train,y_train)
print("Train Error: ",np.sqrt(mean_squared_error(y_train,xgb_Model.predict(X_train))))
y_pred_XGB=xgb_Model.predict(X_test)
XGBoost_RMSE=np.sqrt(mean_squared_error(y_test,y_pred_XGB))
print("Test Error: ",XGBoost_RMSE)
print("---"*108)
print("R2 SCORE:",random_forest_Model.score(x,y))

# **COMPARISON OF RESULTS**

In [ ]:
results=pd.DataFrame({"y_actual":y_test,
               "y_pred_linear":y_pred_linear,
               "y_pred_elasticNET":y_pred_elastic,
               "y_pred_randomForest":y_pred_randomForest,
                      "y_pred_XGBOOST":y_pred_XGB
                     })
results

In [ ]:
models_RMSE=pd.DataFrame({"Models":["Linear Regression","Elastic Net","Random Forest","XGBoost"],
                         "RMSE":[linear_RMSE,elasticNet_RMSE,randomForest_RMSE,XGBoost_RMSE]})
fig = px.bar(models_RMSE, x='RMSE', y='Models',
             labels={'Root Mean Squared Error of Models'},
             hover_data=['RMSE', 'Models'], color='Models',
             height=400)
fig.show()